In [7]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [15]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # input - 28x28x1 output - 22x22x32
        self.conv1 = nn.Sequential(
            # RF - 3x3
            nn.Conv2d(1, 16, 3, bias=False),
            nn.BatchNorm2d(16), # 26
            nn.ReLU(),
            nn.Dropout2d(0.1),

            # RF - 5x5
            nn.Conv2d(16, 16, 3, bias=False), 
            nn.BatchNorm2d(16),# 24
            nn.ReLU(),
            nn.Dropout2d(0.1),

            # RF - 7x7
            nn.Conv2d(16, 32, 3, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout2d(0.1),
        )

        # translation layer
        # input - 22x22x32 output - 11x11x16
        self.trans1 = nn.Sequential(
            # RF - 7x7
            nn.Conv2d(32, 16, 1, bias=False),
            nn.ReLU(),

            # RF - 14x14
            nn.MaxPool2d(2, 2), # 11
        )
        
        # input - 11x11x16 output - 7x7x16
        self.conv2 = nn.Sequential(
            # RF - 16x16
            nn.Conv2d(16, 16, 3, bias=False),
            nn.BatchNorm2d(16), # 9
            nn.ReLU(),
            nn.Dropout2d(0.1),

            # RF - 18x18
            nn.Conv2d(16, 16, 3, bias=False), # 7
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Dropout2d(0.1),
        )

        # input - 7x7x16 output - 5x5x16
        self.conv3 = nn.Sequential(
            # RF - 20x20
            nn.Conv2d(16, 16, 3, padding=1, bias=False), # 5
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Dropout2d(0.1),

            # RF - 22x22
            nn.Conv2d(16, 16, 3, bias=False), # 5
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Dropout2d(0.1),
        )

        # GAP Layer
        # input - 5x5x16 output - 1x1x10
        self.avg_pool = nn.Sequential(
            # # RF - 22x22
            nn.Conv2d(16, 10, 1, bias=False),
            nn.AvgPool2d(5)
        )

    """forward: performs a forward pass when model(x) is called
    Params
        x: the input data
    Returns
        y: the output of the model
    """
    def forward(self, x):
        x = self.conv1(x)
        x = self.trans1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.avg_pool(x)

        x = x.view(-1, 10)
        return F.log_softmax(x)

In [16]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             144
       BatchNorm2d-2           [-1, 16, 26, 26]              32
              ReLU-3           [-1, 16, 26, 26]               0
         Dropout2d-4           [-1, 16, 26, 26]               0
            Conv2d-5           [-1, 16, 24, 24]           2,304
       BatchNorm2d-6           [-1, 16, 24, 24]              32
              ReLU-7           [-1, 16, 24, 24]               0
         Dropout2d-8           [-1, 16, 24, 24]               0
            Conv2d-9           [-1, 32, 22, 22]           4,608
      BatchNorm2d-10           [-1, 32, 22, 22]              64
             ReLU-11           [-1, 32, 22, 22]               0
        Dropout2d-12           [-1, 32, 22, 22]               0
           Conv2d-13           [-1, 16, 22, 22]             512
             ReLU-14           [-1, 16,

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:89: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [17]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [18]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [19]:
# move the model to the specified device
model = Net().to(device)

# use Stochastic Gradient Descent as the optimizer
# learning_rate=0.02 and momentum=0.7 for SGD
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# set the number of epochs to train for
n_epoch = 20

# run it for N_EPOCH number of times
for epoch in range(1, n_epoch+1):
    print('\nEpoch {} : '.format(epoch))
    # train the model
    train(model, device, train_loader, optimizer, epoch)
    # test the model
    test(model, device, test_loader)


Epoch 1 : 


loss=0.5276613235473633 batch_id=468: 100%|██████████| 469/469 [00:22<00:00, 20.46it/s]



Test set: Average loss: 0.1766, Accuracy: 9624/10000 (96%)


Epoch 2 : 


loss=0.1593027412891388 batch_id=468: 100%|██████████| 469/469 [00:23<00:00, 20.27it/s]



Test set: Average loss: 0.0730, Accuracy: 9790/10000 (98%)


Epoch 3 : 


...


Test set: Average loss: 0.0229, Accuracy: 9929/10000 (99%)


Epoch 19 : 


loss=0.09060657769441605 batch_id=468: 100%|██████████| 469/469 [00:23<00:00, 20.07it/s]



Test set: Average loss: 0.0252, Accuracy: 9926/10000 (99%)


Epoch 20 : 


loss=0.11311253160238266 batch_id=468: 100%|██████████| 469/469 [00:23<00:00, 20.14it/s]



Test set: Average loss: 0.0221, Accuracy: 9926/10000 (99%)

